# Impor packages

In [20]:
# pip install pandas
# pip install numpy
# pip install datetime
import pandas as pd
import numpy as np
import datetime as dt

# Impor data dari CSV ke DataFrame

In [21]:
df = pd.read_csv('Online Retail Data.csv', header=0)
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5.0,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1.0,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5.0,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1.0,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1.0,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
57043,499638,22412,METAL SIGN NEIGHBOURHOOD WITCH,1.0,2010-03-01 13:45:00,4.21,NaN
57044,499638,22495,SET OF 2 ROUND TINS CAMEMBERT,1.0,2010-03-01 13:45:00,5.91,NaN
57045,499638,22496,SET OF 2 ROUND TINS DUTCH CHEESE,1.0,2010-03-01 13:45:00,5.91,NaN
57046,499638,22499,WOODEN UNION JACK BUNTING,2.0,2010-03-01 13:45:00,12.72,NaN


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57048 entries, 0 to 57047
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      57048 non-null  object 
 1   product_code  57048 non-null  object 
 2   product_name  56292 non-null  object 
 3   quantity      57047 non-null  float64
 4   order_date    57047 non-null  object 
 5   price         57047 non-null  float64
 6   customer_id   42562 non-null  float64
dtypes: float64(3), object(4)
memory usage: 3.0+ MB


# Data cleansing

In [23]:
import pandas as pd
import numpy as np
from scipy import stats

# Copy DataFrame
df_clean = df.copy()

# Membuat kolom date
df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date

df_clean['date'] = pd.to_datetime(df_clean['date'])  # Pastikan format datetime

# Menghapus semua baris tanpa customer_id
df_clean = df_clean.dropna(subset=['customer_id'])

# Mengonversi customer_id menjadi string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)

# Menghapus semua baris tanpa product_name
df_clean = df_clean.dropna(subset=['product_name'])

# Membuat semua product_name berhuruf kecil
df_clean['product_name'] = df_clean['product_name'].str.lower()

# Menghapus semua baris dengan product_code atau product_name mengandung 'test'
df_clean = df_clean[~df_clean['product_code'].str.lower().str.contains('test', na=False)]
df_clean = df_clean[~df_clean['product_name'].str.contains('test', na=False)]

# Menghapus baris dengan status cancelled (order_id diawali 'C')
df_clean = df_clean[~df_clean['order_id'].astype(str).str.startswith('C')]

# Mengubah nilai quantity yang negatif menjadi positif
df_clean['quantity'] = df_clean['quantity'].abs()

# Menghapus baris dengan price bernilai negatif
df_clean = df_clean[df_clean['price'] > 0]

# Membuat nilai amount (perkalian antara quantity dan price)
df_clean['amount'] = df_clean['quantity'] * df_clean['price']

# Mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu yang paling sering muncul
most_freq_product_name = (
    df_clean.groupby(['product_code', 'product_name'])
    .agg(order_cnt=('order_id', 'nunique'))
    .reset_index()
    .sort_values(['product_code', 'order_cnt'], ascending=[True, False])
)

most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank'] == 1].drop(columns=['order_cnt', 'rank'])

df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name': 'most_freq_product_name'}), on='product_code', how='left')
df_clean['product_name'] = df_clean['most_freq_product_name'].fillna(df_clean['product_name'])
df_clean = df_clean.drop(columns='most_freq_product_name')

# Menghapus outlier berdasarkan z-score
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity', 'amount']])) < 3).all(axis=1)]

# Reset index
df_clean = df_clean.reset_index(drop=True)

df_clean


<ipython-input-23-996b0ded607c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['customer_id'] = df_clean['customer_id'].astype(str)


,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,amount
0,493414,21844,retro spot mug,36.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,91.80
1,493414,21533,retro spot large milk jug,12.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,51.00
2,493414,37508,new england ceramic cake server,2.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,5.10
3,493414,35001G,hand open shape gold,2.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,8.50
4,493414,21527,retro spot traditional teapot,12.0,2010-01-04 10:28:00,6.95,14590.0,2010-01-04,83.40
...,...,...,...,...,...,...,...,...,...
40870,499636,85231E,strawberry scented set/9 t-lights,12.0,2010-03-01 13:41:00,0.85,13050.0,2010-03-01,10.20
40871,499636,85214,tub 24 pink flower pegs,12.0,2010-03-01 13:41:00,1.65,13050.0,2010-03-01,19.80
40872,499636,21363,home small wood letters,3.0,2010-03-01 13:41:00,4.95,13050.0,2010-03-01,14.85
40873,499636,21232,strawberry ceramic trinket box,12.0,2010-03-01 13:41:00,1.25,13050.0,2010-03-01,15.00


In [24]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40875 entries, 0 to 40874
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      40875 non-null  object        
 1   product_code  40875 non-null  object        
 2   product_name  40875 non-null  object        
 3   quantity      40875 non-null  float64       
 4   order_date    40875 non-null  object        
 5   price         40875 non-null  float64       
 6   customer_id   40875 non-null  object        
 7   date          40875 non-null  datetime64[ns]
 8   amount        40875 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 2.8+ MB


# Menyiapkan data basket

## Buat DataFrame basket

In [25]:
basket = pd.pivot_table(df_clean, index='order_id', columns='product_name', values='product_code', aggfunc='nunique', fill_value=0)
basket

product_name,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 mini toadstool pegs,12 pencils small tube posy,12 pencils small tube red spotty,12 pencils small tube skull,12 pencils tall tube posy,12 pencils tall tube red spotty,12 pencils tall tube skulls,...,you're confusing me metal sign,zinc finish 15cm planter pots,zinc heart lattice 2 wall planter,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499629,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
499630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
499632,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1869 entries, 493414 to 499636
Columns: 2648 entries, 12 daisy pegs in wood box to zinc willie winkie  candle stick
dtypes: int64(2648)
memory usage: 37.8+ MB


## Encode DataFrame basket dengan nilai True untuk semua nilai di atas 0 dan False untuk semua nilai 0

In [27]:
def encode(x):
    if x==0:
        return False
    if x>0:
        return True

basket_encode = basket.applymap(encode)
basket_encode

<ipython-input-27-31958d44448d>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encode = basket.applymap(encode)


product_name,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 mini toadstool pegs,12 pencils small tube posy,12 pencils small tube red spotty,12 pencils small tube skull,12 pencils tall tube posy,12 pencils tall tube red spotty,12 pencils tall tube skulls,...,you're confusing me metal sign,zinc finish 15cm planter pots,zinc heart lattice 2 wall planter,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499629,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
499630,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
499632,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [28]:
basket_encode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1869 entries, 493414 to 499636
Columns: 2648 entries, 12 daisy pegs in wood box to zinc willie winkie  candle stick
dtypes: bool(2648)
memory usage: 4.7+ MB


## Ambil transaksi dengan banyaknya produk unik lebih dari 1 saja

In [29]:
basket_filter = basket_encode[(basket_encode>0).sum(axis=1)>1]
basket_filter

product_name,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 mini toadstool pegs,12 pencils small tube posy,12 pencils small tube red spotty,12 pencils small tube skull,12 pencils tall tube posy,12 pencils tall tube red spotty,12 pencils tall tube skulls,...,you're confusing me metal sign,zinc finish 15cm planter pots,zinc heart lattice 2 wall planter,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499628,False,False,False,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
499629,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
499632,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
basket_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1743 entries, 493414 to 499636
Columns: 2648 entries, 12 daisy pegs in wood box to zinc willie winkie  candle stick
dtypes: bool(2648)
memory usage: 4.4+ MB
